<a href="https://colab.research.google.com/github/Ithomas13/ML-Earnings-Predictor/blob/main/Trading_Insights_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install and import necessary libraries
!pip install yfinance
!pip install scipy
!pip install statsmodels
!pip install ipywidgets
!pip install --upgrade nbformat

# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import statsmodels.api as sm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Create a text input widget for the ticker symbol
ticker_input = widgets.Text(
    value='AAPL',
    placeholder='Enter ticker symbol',
    description='Ticker:',
    disabled=False
)

# Display the ticker input widget
display(ticker_input)

# Instructions for earnings data entry
display(HTML("""
    <h3>Enter Earnings Data</h3>
    <p>Please paste the earnings dates and corresponding EPS values below.</p>
    <p><strong>Format:</strong> Each line should contain the date and EPS separated by a comma.</p>
    <p><strong>Example:</strong></p>
    <pre>
2023-08-03, 1.26
2023-05-04, 1.52
2023-02-02, 1.88
    </pre>
    <p>You can use the following Apple earnings data by copying and pasting it into the field below:</p>
    <pre>
2023-08-03, 1.26
2023-05-04, 1.52
2023-02-02, 1.88
2022-10-27, 1.29
2022-07-28, 1.20
2022-04-28, 1.52
2022-01-27, 2.10
2021-10-28, 1.24
2021-07-27, 1.30
2021-04-28, 1.40
    </pre>
"""))

# Create a textarea widget for earnings data input
earnings_data_input = widgets.Textarea(
    value='',
    placeholder='Paste earnings data here in YYYY-MM-DD, EPS format...',
    description='Earnings Data:',
    layout=widgets.Layout(width='100%', height='200px'),
    disabled=False
)

# Display the earnings data input widget
display(earnings_data_input)

# Create a button to run the analysis
run_button = widgets.Button(description="Run Analysis", button_style='success')

# Display the run button
display(run_button)

# Define the function to perform the analysis
def run_analysis_function(b):
    clear_output()
    display(ticker_input, earnings_data_input, run_button)

    # Get the ticker symbol and ensure it's uppercase
    ticker = ticker_input.value.strip().upper()
    if not ticker:
        print("Please enter a valid ticker symbol.")
        return
    print(f"Fetching data for {ticker}...")

    # Fetch stock data
    try:
        stock_data = yf.download(ticker, start='2018-01-01', end=datetime.today().strftime('%Y-%m-%d'))
        if stock_data.empty:
            print("No stock data found for this ticker.")
            return
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return

    # Parse earnings data from the textarea
    earnings_text = earnings_data_input.value.strip()
    if not earnings_text:
        print("Please enter earnings data in the specified format.")
        return

    earnings_list = []
    lines = earnings_text.split('\n')
    for line in lines:
        if line.strip() == '':
            continue  # Skip empty lines
        try:
            date_str, eps_str = line.split(',')
            date = pd.to_datetime(date_str.strip())
            eps = float(eps_str.strip())
            earnings_list.append({'Earnings Date': date, 'EPS': eps})
        except ValueError:
            print(f"Invalid format in line: '{line}'. Please use 'YYYY-MM-DD, EPS'.")
            return

    earnings_df = pd.DataFrame(earnings_list)
    if earnings_df.empty:
        print("No valid earnings data entered.")
        return

    earnings_df.sort_values('Earnings Date', inplace=True)
    earnings_df.reset_index(drop=True, inplace=True)

    # Display the entered earnings data for verification
    print("\nEntered Earnings Data:")
    display(earnings_df)

    # Fetch VIX data
    try:
        vix_data = yf.download('^VIX', start='2018-01-01', end=datetime.today().strftime('%Y-%m-%d'))
        if vix_data.empty:
            print("No VIX data found.")
            return
    except Exception as e:
        print(f"Error fetching VIX data: {e}")
        return

    # Calculate price changes around earnings dates
    price_changes = []
    for index, row in earnings_df.iterrows():
        date = row['Earnings Date']
        # Find the closest previous trading day
        before_date = stock_data[:date - timedelta(days=1)].tail(1).index
        # Find the closest next trading day
        after_date = stock_data[date + timedelta(days=1):].head(1).index

        if len(before_date) > 0 and len(after_date) > 0:
            before_price = stock_data.loc[before_date[0], 'Close']
            after_price = stock_data.loc[after_date[0], 'Close']
            change = ((after_price - before_price) / before_price) * 100
            price_changes.append({
                'Earnings Date': date,
                'Price Change (%)': change,
                'EPS': row['EPS']
            })
        else:
            print(f"Data missing for earnings date: {date.date()}")

    price_changes_df = pd.DataFrame(price_changes)
    if price_changes_df.empty:
        print("No price change data calculated.")
        return

    # Display the calculated price changes
    print("\nCalculated Price Changes Around Earnings Dates:")
    display(price_changes_df)

    # Get VIX level at earnings dates
    vix_levels = []
    for date in price_changes_df['Earnings Date']:
        # Fetch the VIX closing price on the day before earnings
        vix_before = vix_data[:date].tail(1)
        if not vix_before.empty:
            vix_close = vix_before['Close'].values[0]
        else:
            vix_close = np.nan
        vix_levels.append(vix_close)
    price_changes_df['VIX Level'] = vix_levels

    # Drop rows with missing values
    analysis_df = price_changes_df.dropna()
    if analysis_df.empty:
        print("Not enough data after removing missing values.")
        return

    # Display the analysis DataFrame
    print("\nAnalysis DataFrame:")
    display(analysis_df)

    # Independent variables (add a constant term for intercept)
    X = analysis_df[['EPS', 'VIX Level']]
    X = sm.add_constant(X)

    # Dependent variable
    Y = analysis_df['Price Change (%)']

    # Perform linear regression
    try:
        model = sm.OLS(Y, X).fit()
    except Exception as e:
        print(f"Error performing regression: {e}")
        return

    # Display regression results
    print("\nRegression Model Summary:")
    print(model.summary())

    # Get latest EPS (use the last entered EPS)
    latest_estimate = earnings_df['EPS'].iloc[-1]

    # Get latest VIX level
    latest_vix = vix_data['Close'].iloc[-1]

    # Prepare input data for prediction
    input_data = pd.DataFrame({
        'const': [1],
        'EPS': [latest_estimate],
        'VIX Level': [latest_vix]
    })

    # Predict price change
    try:
        predicted_change = model.predict(input_data)[0]
    except Exception as e:
        print(f"Error making prediction: {e}")
        return
    print(f"\nPredicted Price Change After Next Earnings: {predicted_change:.2f}%")

    # Provide trade suggestion
    if predicted_change > 0:
        prediction = 'increase'
        trade_action = 'Consider buying shares before the earnings date.'
    else:
        prediction = 'decrease'
        trade_action = 'Consider selling shares or holding off on buying before the earnings date.'

    print(f"\nThe stock price is likely to {prediction} after the next earnings report.")
    print(trade_action)

    # Display the data
    print("\nHistorical Price Changes Around Earnings Dates:")
    display(price_changes_df[['Earnings Date', 'Price Change (%)', 'EPS', 'VIX Level']])

    # Plotting Price Change vs. Earnings Date with Correct Bar Chart
    plt.figure(figsize=(14, 7))
    plt.bar(price_changes_df['Earnings Date'].dt.strftime('%Y-%m-%d'), price_changes_df['Price Change (%)'], color='skyblue')
    plt.xlabel('Earnings Date')
    plt.ylabel('Price Change (%)')
    plt.title(f'Stock Price Changes After Earnings for {ticker}')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Link the button to the analysis function
run_button.on_click(run_analysis_function)

# Display the button
display(run_button)


Text(value='AAPL', description='Ticker:', placeholder='Enter ticker symbol')

Textarea(value='2023-08-03, 1.26\n2023-05-04, 1.52\n2023-02-02, 1.88\n2022-10-27, 1.29\n2022-07-28, 1.20\n2022…

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Fetching data for AAPL...


[*********************100%***********************]  1 of 1 completed

...


Calculated Price Changes Around Earnings Dates:


,Earnings Date,Price Change (%),EPS
0,2021-04-28,Ticker AAPL -0.677154 dtype: float64,1.40
1,2021-07-27,Ticker AAPL -2.691447 dtype: float64,1.30
2,2021-10-28,Ticker AAPL 0.638221 dtype: float64,1.24
3,2022-01-27,Ticker AAPL 6.662891 dtype: float64,2.10
4,2022-04-28,Ticker AAPL 0.689779 dtype: float64,1.52
5,2022-07-28,Ticker AAPL 3.648204 dtype: float64,1.20
6,2022-10-27,Ticker AAPL 4.278532 dtype: float64,1.29
7,2023-02-02,Ticker AAPL 6.236681 dtype: float64,1.88
8,2023-05-04,Ticker AAPL 3.654827 dtype: float64,1.52
9,2023-08-03,Ticker AAPL -5.499006 dtype: float64,1.26



Analysis DataFrame:


,Earnings Date,Price Change (%),EPS,VIX Level
0,2021-04-28,Ticker AAPL -0.677154 dtype: float64,1.40,[17.280000686645508]
1,2021-07-27,Ticker AAPL -2.691447 dtype: float64,1.30,[19.360000610351562]
2,2021-10-28,Ticker AAPL 0.638221 dtype: float64,1.24,[16.530000686645508]
3,2022-01-27,Ticker AAPL 6.662891 dtype: float64,2.10,[30.489999771118164]
4,2022-04-28,Ticker AAPL 0.689779 dtype: float64,1.52,[29.989999771118164]
5,2022-07-28,Ticker AAPL 3.648204 dtype: float64,1.20,[22.329999923706055]
6,2022-10-27,Ticker AAPL 4.278532 dtype: float64,1.29,[27.389999389648438]
7,2023-02-02,Ticker AAPL 6.236681 dtype: float64,1.88,[18.729999542236328]
8,2023-05-04,Ticker AAPL 3.654827 dtype: float64,1.52,[20.09000015258789]
9,2023-08-03,Ticker AAPL -5.499006 dtype: float64,1.26,[15.920000076293945]


Error performing regression: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
